### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data
2. Соберем пайплайн с простейшим препроцессингом на числовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Загрузим данные

In [ ]:
df = pd.read_csv("./heart_failure_clinical_records_dataset.csv")
df.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1


Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['DEATH_EVENT'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [ ]:
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [ ]:
continuos_cols = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time']
base_cols = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']

target = 'DEATH_EVENT'

Соберем кусок, ответственный за feature engineering

In [ ]:
continuos_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [ ]:
feats = FeatureUnion(continuos_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 1.27993208e+00, -8.12177301e-04,  5.49747992e-01, ...,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 3.91809798e-01, -4.98735254e-01, -1.16687337e+00, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 2.16805436e+00, -5.94285525e-01,  5.49747992e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       ...,
       [-4.96312483e-01,  1.75424989e-01,  5.49747992e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.47324699e+00, -8.12177301e-04, -7.37718032e-01, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.72399322e+00,  3.34675440e-01, -1.16687337e+00, ...,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00]])

Добавим простейший классификатор

In [ ]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 45.9 ms, sys: 0 ns, total: 45.9 ms
Wall time: 52.8 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('creatinine_phosphokinase',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='creatinine_phosphokinase')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('ejection_fraction',
                                                 Pipeline(steps=[('sele

Посмотрим, как выглядит наш pipeline

In [ ]:
pipeline.steps

[('features', FeatureUnion(transformer_list=[('age',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='age')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('creatinine_phosphokinase',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='creatinine_phosphokinase')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('ejection_fraction',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='ejection_fraction'...
                                  Pipeline(steps=[('selector',
                                  

Сохраним модель (пайплайн)

In [ ]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)